# PostNord - Logistics dataset - Cleaning then vizualising (tablau)

In [90]:
import pandas as pd
import numpy as np
import os
import glob
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
sns.set_palette('bright')
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm
import glob
import dask.dataframe as dd
import dask
import pandas as pd
import warnings
import glob
import warnings
import pandas as pd
from concurrent.futures import ThreadPoolExecutor

In [91]:
# Ignore warnings for clean output
warnings.filterwarnings("ignore")

# Define a function to read and concatenate files
def read_and_concat_files(file_pattern):
    file_paths = glob.glob(file_pattern)
    with ThreadPoolExecutor() as executor:
        df_list = list(executor.map(pd.read_excel, file_paths))
    return pd.concat(df_list, ignore_index=True)

# File patterns for each category
file_patterns = {
    "Groupage": "C:\\Users\\tgsog\\OneDrive\\Desktop\\Visual Analytics\\Exam\\exam docs\\Groupage\\DatadumpProduct G*.xls",
    "Innight": "C:\\Users\\tgsog\\OneDrive\\Desktop\\Visual Analytics\\Exam\\exam docs\\Innight\\DatadumpProduct E*.xls",
    "Pallet": "C:\\Users\\tgsog\\OneDrive\\Desktop\\Visual Analytics\\Exam\\exam docs\\Pallet\\DatadumpProduct P*.xls",
    "Road_Freight": "C:\\Users\\tgsog\\OneDrive\\Desktop\\Visual Analytics\\Exam\\exam docs\\Road_Freight\\DatadumpProduct R_*.xls"
}

# Process each category
pGdf = read_and_concat_files(file_patterns["Groupage"])
pIdf = read_and_concat_files(file_patterns["Innight"])
pPdf = read_and_concat_files(file_patterns["Pallet"])
pRdf = read_and_concat_files(file_patterns["Road_Freight"])

# Load additional static files (assuming these are smaller and don't need parallel processing)
md_df = pd.read_excel("C:\\Users\\tgsog\\OneDrive\\Desktop\\Visual Analytics\\Exam\\exam docs\\Master data\\ADS_DDS_RF_CustomerNumbers_MasterData.xlsx")
rf_df = pd.read_excel("C:\\Users\\tgsog\\OneDrive\\Desktop\\Visual Analytics\\Exam\\exam docs\\Master data\\ADS_DDS_RF_CustomerNumbers_MasterData - road freight.xlsx")

md_df.columns = ['Customer_ID', 'Main Customer', 'Industry Level 1', 'Industry Level 2','System', 'Allocated SAP Number']

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but 

### making a column names list for all geo data

In [92]:
geo_columns = ["country code",     #: iso country code, 2 characters
"postal code",       #: varchar(20)
"place name",        #: varchar(180)
"admin name1",       #: 1. order subdivision (state) varchar(100)
"admin code1",       #: 1. order subdivision (state) varchar(20)
"admin name2",       #: 2. order subdivision (county/province) varchar(100)
"admin code2",       #: 2. order subdivision (county/province) varchar(20)
"admin name3",       #: 3. order subdivision (community) varchar(100)
"admin code3",       #: 3. order subdivision (community) varchar(20)
"latitude",          #: estimated latitude (wgs84)
"longitude",         #: estimated longitude (wgs84)
"accuracy"]          #: accuracy of lat/lng from 1=estimated, 4=geonameid, 6=centroid of addresses or shape

### Loading geo data - and concatinating

In [93]:
gn_no = pd.read_csv("C:\\Users\\tgsog\\OneDrive\\Desktop\\Visual Analytics\\Geonames data\\NO.csv",header = None, names = geo_columns,sep = "\t")
gn_dk =pd.read_csv("C:\\Users\\tgsog\\OneDrive\\Desktop\\Visual Analytics\\Geonames data\\DK.csv",header = None,names = geo_columns, sep = "\t")
gn_se = pd.read_csv("C:\\Users\\tgsog\\OneDrive\\Desktop\\Visual Analytics\\Geonames data\\SE.csv", header = None, names =geo_columns ,sep = "\t")
gn_se["postal code"] =gn_se["postal code"].str.replace(" ","")
geo_df = pd.concat([gn_no,gn_dk,gn_se])
geo_df["postal code"] = geo_df["postal code"].astype("int")

## DDS groupage - cleaning

In [94]:
#Filling NAs and removing PKK
pGdf["Parcel/Pallet type"] =pGdf["Parcel/Pallet type"].fillna("PLL")
pGdf["Parcel/Pallet type"] = pGdf["Parcel/Pallet type"].replace("PKK","SPL")

#Remove columns
pGdf = pGdf.drop(columns=["Base_Price","Freight calc_ status"])

#Exclude parcels above 2500KG
pGdf =pGdf[pGdf["ConsignorWeight"]<=2500]

## DDS innight - cleaning

In [95]:
#Filling NAs and removing PKK
pIdf["Parcel/Pallet type"] =pIdf["Parcel/Pallet type"].fillna("PLL")
pIdf["Parcel/Pallet type"] = pIdf["Parcel/Pallet type"].replace("PKK","SPL")

#Remove columns
pIdf = pIdf.drop(columns=["Base_Price","Freight calc_ status"])

## DDS pallet - cleaning

In [96]:
#Filling NAs and removing PKK
pPdf["Parcel/Pallet type"] =pPdf["Parcel/Pallet type"].fillna("PLL")
pPdf["Parcel/Pallet type"] = pPdf["Parcel/Pallet type"].replace("PKK","SPL")

#Remove columns
pPdf = pPdf.drop(columns=["Base_Price","Freight calc_ status"])

pPdf = pPdf[pPdf["ConsignorWeight"]<=1000]

## DDS road_freight - cleaning

In [97]:
#Filling NAs and removing PKK
pRdf["Parcel/Pallet type"] =pRdf["Parcel/Pallet type"].fillna("PLL")
pRdf["Parcel/Pallet type"] = pRdf["Parcel/Pallet type"].replace("PKK","SPL")

#Remove columns
pRdf = pRdf.drop(columns=["Base_Price","Freight calc_ status"])

## Concatinate product data

In [98]:
df  = pd.concat([pRdf,pPdf, pIdf,pGdf])

## Exclude duplicate shipping numbers (should be unique per order)

In [99]:
df = df.dropna(subset ="Shipment Tracking Number")

## Left Join: masterdata on product data

In [100]:
df = df.merge(md_df, left_on="Customer_ID", right_on="Customer_ID")

## Left Join: geo data on product data (for both consignor / consignees)

In [101]:
#Making new column names for the consignor, consignee - additions
#Consignor list of columns
dk_l =[]
for x in gn_dk.columns:
    dk_l.append("consigner_"+x)
    
#consignee list of columns
geo_l =[]
for x in geo_df.columns:
    geo_l.append("conignee_"+x)
    
#Assigning column names
gn_dk.columns = dk_l
geo_df.columns = geo_l

In [102]:
#Consignor-mergning on location master data files
df =df.merge(gn_dk, left_on =["ConsignorCountryCode","Consignor Location Nr"], right_on =["consigner_country code", "consigner_postal code"])

#Consignee-mergning on location master data files
df = df.merge(geo_df, left_on = ["ConsigneeCountryCode","Consignee Location Nr"],right_on =["conignee_country code","conignee_postal code"])

## Making sure there are no shipment NA, and no duplicated rows

In [103]:
df = df.dropna(subset ="Shipment Tracking Number")
df = df.drop_duplicates()

# Cleaned dataset for PostNord

In [104]:
df.head()

,Customer_ID,ConsignorCountryCode,Consignor Location Nr,ConsigneeCountryCode,Consignee Location Nr,Product,Parcel/Pallet type,Shipment Tracking Number,ShipmentDate,ConsignorParcelCount,...,conignee_place name,conignee_admin name1,conignee_admin code1,conignee_admin name2,conignee_admin code2,conignee_admin name3,conignee_admin code3,conignee_latitude,conignee_longitude,conignee_accuracy
0,EAAAAAEIE,DK,4690,NO,951,R,PLL,5DO5Z8COO5C44A686,2019-08-13,1,...,Oslo,Oslo County,12,Oslo,301.0,NaN,NaN,59.9127,10.7461,4.0
1,HDHIAAABH,DK,8270,NO,566,R,PLL,5DO5Z8COO5C8A4ODD,2019-08-20,33,...,Oslo,Oslo County,12,Oslo,301.0,NaN,NaN,59.9127,10.7461,4.0
2,HDHIAAABH,DK,8270,NO,566,R,PLL,5DO5Z8COO5C8DZ5B6,2019-08-21,33,...,Oslo,Oslo County,12,Oslo,301.0,NaN,NaN,59.9127,10.7461,4.0
3,HDHIAAABH,DK,8270,NO,566,R,PLL,5DO5Z8COO5CZ58C5A,2019-08-22,33,...,Oslo,Oslo County,12,Oslo,301.0,NaN,NaN,59.9127,10.7461,4.0
4,HDHIAAABH,DK,8270,NO,566,R,PLL,5DO5Z8COO54OBCBB5,2019-08-23,33,...,Oslo,Oslo County,12,Oslo,301.0,NaN,NaN,59.9127,10.7461,4.0


In [ ]:
# Writing dataset

In [ ]:
df.to_csv("C:\Users\tgsog\OneDrive\Desktop\\Visual Analytics\\Exam\\PostNordData.csv", index=False)
